In [1]:
# Run this block first!
import pandas as pd
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

from scipy.spatial.distance import cosine
from sklearn.metrics import pairwise_distances

In [2]:
# Steps to get the data:
# 1. Go to the link: https://drive.google.com/drive/folders/1ZNKHlrBxqbePdNElohyMtVgrWNaKFQhD?usp=sharing
# 2. Click on the folder name and click to `Copy to Drive` option (copy to root folder!)
# 3. Then give permissions after running this line
# 4. After giving permission, run the next line to copy files to your collab session.

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


# <font color='#FFBD33'>**Assignment 6 - Recommend Song by Mood**</font> 

This is the <font color='cyan'>Assignment 6</font> for the LING360 - Computational Methods in Lingustics course and it is worth a total of  <font color='cyan'>**10 points** + **5 points (Bonus)**</font>.

In this assignment, we are going to create a novel application, which is a song recommender website which will recommend a list of songs generated on the spot using the emotion detection model and text similarity.

Our general plan is to:
1. Train an **emotion detection model**
2. Find emotions in **a song corpus**
3. Implement **web app** which can recommend songs by looking at the **similarity between input text and songs**. Similarity metrics are going to be the
    1. Emotion vector similarity    
    1. Count vector similarity

The topics include:
1. Naive Bayes
2. Streamlit


There's a total of  <font color='cyan'>**2 main tasks**</font> and <font color='cyan'>**x subs tasks**</font>. For each task, please write your code between the following lines:

```
## YOUR CODE STARTS



## YOUR CODE ENDS
```

Before working on the assignment, please copy this notebook to your own drive. You can use ```Save a copy in Drive``` under the ```File``` menu on top left.

Please, run every cell in your code to make sure that it works properly before submitting it. 

Once you are ready to submit, download two versions of your code:

*   Download .ipynb
*   Download .py

These are both available under the ```File``` menu on top left. 

Then, compress your files (zip, rar, or whatever) and upload the compressed file to Moodle.

If you have any questions, please contact with karahan.sahin@boun.edu.tr


In [4]:
! cp "/content/drive/MyDrive/Assignment 6/main.py" -t .
! cp "/content/drive/MyDrive/Assignment 6/song_dataset.csv" -t .
! cp "/content/drive/MyDrive/Assignment 6/train_emotion.csv" -t .
! cp "/content/drive/MyDrive/Assignment 6/test_emotion.csv" -t .

## <font color='#FFBD33'>**Q1:** Emotion Classifier</font> `10 points`

Train a model for emotion recognition using Naive Bayes Classifiers

### <font color='#FFBD33'>Q1.1: Import and Clean Data</font> `2 points`

First we need to get our data and preprocess it accordingly.

<font color='#FFBD33'>**Instructions:**</font>

1. Import train and test sets namely, `train_emotion.csv` and `test_emotion.csv`.
2. Read line by line to get (sentence, label) pairs by separating each line with `\t` character.
2. For training data, add sentences to a variable called `train_sentences` and add labels to a variable called `train_labels`.
2. For test data, add sentences to a variable called `test_sentences` and add labels to a variable called `test_labels`.
3. Then translate the numeric labels into names `{ 0: "sadness", 1: "joy" , 2:  "love" , 3: "anger", 4: "fear", 5: 'surprise'}` for train and test labels.

<font color='#FFBD33'>**Notes:**</font>

1. Don't forget to open your file with `encoding="utf-8"` parameter.

In [8]:
## YOUR CODE STARTS

train_file = "train_emotion.csv"
test_file = "test_emotion.csv"

train_sentences = []
train_labels = []
test_sentences = []
test_labels = []

label_mapping = {"sadness": 0, "joy": 1, "love": 2, "anger": 3, "fear": 4, "surprise": 5}

with open(train_file, "r", encoding="utf-8") as file:
    for line in file:
        sentence, label = line.strip().split("\t")
        train_sentences.append(sentence)
        train_labels.append(label_mapping[label])

with open(test_file, "r", encoding="utf-8") as file:
    for line in file:
        sentence, label = line.strip().split("\t")
        test_sentences.append(sentence)
        test_labels.append(label_mapping[label])


## YOUR CODE ENDS

### <font color='#FFBD33'>Q1.2: Train Model/Vectorizer</font> `3 points`

Process the train and test datasets and train your Naive Bayes model with train set.

<font color='#FFBD33'>**Instructions:**</font>

2. Then using `bow.fit_transform()` method, fit the `train_sentences` and assign it into a variable called `X_train`
3. Generate an instance of `MultinomialNB()` object, and assign it into a variable called `model`
4. Then using `model.fit()` method, fit the `X_train` and `train_labels`.
5. Finally run the last line to see your model results.

In [10]:
## YOUR CODE STARTS

bow = CountVectorizer()
X_train = bow.fit_transform(train_sentences)

model = MultinomialNB()

model.fit(X_train, train_labels)
## YOUR CODE ENDS

MultinomialNB()

In [11]:
X_test = bow.transform(test_sentences)
y_pred = model.predict(X_test)
print(classification_report(test_labels, y_pred))

              precision    recall  f1-score   support

           0       0.73      0.93      0.82      1959
           1       0.71      0.96      0.81      2192
           2       0.88      0.21      0.34       547
           3       0.91      0.55      0.68       912
           4       0.86      0.51      0.64       749
           5       1.00      0.02      0.05       241

    accuracy                           0.74      6600
   macro avg       0.85      0.53      0.56      6600
weighted avg       0.78      0.74      0.71      6600



### <font color='#FFBD33'>Q1.3: Run on Song Corpus</font> `2 points`

Use emotion detection to find the emotions of songs in corpus. Using that model, we are going to emotion features for each song.

<font color='#FFBD33'>**Instructions:**</font>

1. Import train and test sets namely, `song_dataset.csv`.
2. Read line by line to get (song_name, lyrics) pairs by separating each line with `\t` character
2. For song data, add song_name to a variable called `song_titles` and add lyrics to a variable called `song_lyrics`.
3. Then using `bow.transform()` method, fit the `song_lyrics` and assign it into a variable called `X_text`
4. Then using `model.predict_proba()` method, predict the `song_lyrics` and assign it into variable called `X_mood`

In [12]:
example_sentences = [
    "I don't know you, but it will be okay", 
    "I am so angry today", 
    "Really sad day:( "
]
X_example = bow.transform(example_sentences)
print(model.predict_proba(X_example))

[[4.21170078e-01 5.21180504e-01 1.34826548e-02 2.36264742e-02
  2.04338778e-02 1.06410642e-04]
 [4.06675570e-01 1.03311494e-01 6.89331113e-03 4.36998612e-01
  4.45664929e-02 1.55452033e-03]
 [7.32605983e-01 1.32374041e-01 5.08793669e-03 7.07038448e-02
  5.69751005e-02 2.25309388e-03]]


As you can see, now our feature vectors are become the emotion scores extracted from the trained Naive Bayes Model

|sentence       |   anger      |   fear       |      joy       |     love       |    sadness     |   surprise    |
|--------------|--------------|--------------|----------------|----------------|----------------|---------------|
|I don't know you, but it will be okay|2.35078678e-02|2.04108623e-02| 5.20536354e-01 | 1.34679763e-02 | 4.21970631e-01 | 1.06308399e-04|
|I am so angry today|4.37101484e-01|4.45618521e-02| 1.03295068e-01 | 6.89272322e-03 | 4.06594385e-01 | 1.55448722e-03|
|Really sad day:( |7.06614679e-02|5.69343769e-02| 1.32273982e-01 | 5.08437190e-03 | 7.32794178e-01 | 2.25162335e-03|

In [61]:
## YOUR CODE STARTS

import csv

song = "song_dataset.csv"

song_titles = []
song_lyrics = []

with open(song, "r", encoding="utf-8") as file:
    reader = csv.reader(file, delimiter="\t")
    for line in reader:
        song_name, lyrics = line
        song_titles.append(song_name)
        song_lyrics.append(lyrics)

X_text = bow.transform(song_lyrics)

X_mood = model.predict_proba(X_text)
print(X_mood)

## YOUR CODE ENDS

[[7.45387823e-01 2.54612177e-01 8.85658367e-34 2.14061641e-20
  5.60192059e-29 9.55787264e-68]
 [7.93720318e-04 9.99059590e-01 1.46689298e-04 7.52722096e-10
  9.15043217e-14 7.84270612e-33]
 [6.11486922e-05 9.99938851e-01 2.07527598e-14 4.24832471e-13
  1.79697776e-21 1.00558100e-39]
 ...
 [5.04655821e-04 9.99447987e-01 1.45247034e-07 4.71797850e-05
  3.21126255e-08 6.26063331e-12]
 [1.48189482e-03 5.70847244e-01 3.93823231e-01 9.04482643e-03
  2.48028031e-02 1.92763207e-26]
 [1.78062355e-14 1.00000000e+00 1.85331562e-31 3.68633452e-23
  5.97392970e-35 3.68988642e-96]]


### <font color='#FFBD33'>Q1.4: Recommend Song with Metrics</font> `2 points`

Calculate scores of song corpus and get highest scoring topK songs for given sentence.

<font color='#FFBD33'>**Instructions:**</font>

1. First transform your text into bow features and assign it into a variable called `x_text`
2. Then predict its emotion features using `.predict_proba()` and assign it into a variable called `x_mood`
3. Then extract the similarities emotion scores `input_text`, namely `x_mood` between all songs `X_mood`  using `calculateSimilarity()` and assign it to variable called `sim_mood`
3. Then extract the similarities bow scores `input_text`, namely `x_text` between all songs `X_text`  using `calculateSimilarity()` and assign it to variable called `sim_text`
4. The create an empty list called `mean_scores`
5. Then using `zip()`, zip mood and text scores, and iterate over scores to calculate mean score
    ```python
    list1 = [1,2,3]
    list2 = [4,5,6]
    list(zip(list1, list2))
    # Output: [ (1,4), (2,5), (3,6) ] 
    ```
 
6. Then zip the song_names and mean scores, turn the zip object into a dictionary, and sort them by the descending score values.
7. Finally return the `topk` keys of `sorted_scores` dictionary.

In [16]:
def calculateSimilarity(document_vector, all_vectors):
    """The function that checks similarity between input and all instance of features of corpus
    
    """
    return (1 - pairwise_distances(all_vectors,document_vector, metric="cosine")).T[0]

In [50]:
def getSongRecommendation(text, topK=10):
    """Function returns a 
    
    :args:
        text (str): text of explaining current mood
        topK (int): number of song titles to be returned
        
    :returns:
        top_tracks (list): list containing `topK` number of song titles
    """
    
    ## YOUR CODE STARTS
    
    x_text = bow.transform([text])
    x_mood = model.predict_proba(x_text)

    sim_mood = calculateSimilarity(x_mood, X_mood)
    sim_text = calculateSimilarity(x_text, X_text)
    
    mean_scores = []
    
    for mood_score, text_score in list(zip(sim_mood, sim_text)):
      mean_score = (mood_score + text_score) / 2
      mean_scores.append(mean_score)
    
    sorted_scores = sorted(zip(song_titles, mean_scores), key=lambda x: x[1], reverse=True)
    top_tracks = sorted_scores[:topK]

    ## YOUR CODE ENDS
    
    return top_tracks

In [51]:
assert getSongRecommendation("""Very angry, want to punch everything""") == [
    ('I Want To Be Old - The Cure', 0.4865881400301677), 
    ("Somebody's Everything - Dolly Parton", 0.46629072991070464), 
    ("Don't Wanna - Electric Light Orchestra", 0.46627239736234016), 
    ('Cross Oceans - First Aid Kit', 0.46423834544262976), 
    ('Kill Everybody - Skrillex', 0.45675276421899946), 
    ('Stand By - Jeremy Camp', 0.4564354645876385), 
    ('Without Reason - The Fray', 0.4543368996115371), 
    ('Slug - U2', 0.451413762608203), 
    ('Sundress - Ben Kweller', 0.4478342947514802), 
    ('Rock Star - Everclear', 0.4448005087202802)
  ]

AssertionError: ignored

## <font color='#FFBD33'>**BONUS:** Show on Streamlit</font> `5 points`

Use Streamlit to showcase what your model can do! The design is shown below:

<img src="https://lh3.googleusercontent.com/drive-viewer/AFGJ81ogzqu5cYKmGwe439OsZpByzxQqU5yxLl9_89Wo1IEcC0GOfzR7iW7dCq_mLQsG4Tr6nnFQZF40iYuWl4Zs1EBUQLIxPA=s1600" width="400" height="300"/>

<font color='#FFBD33'>**Instructions:**</font>

1. Click on `main.py` file
1. Run lines below.
1. Then generate user interface according to the instructions in the file and the mockup above.

In [67]:
;!pip install streamlit pyngrok --quiet

/bin/bash: -c: line 0: syntax error near unexpected token `"install streamlit pyngrok --quiet"'
/bin/bash: -c: line 0: `pip("install streamlit pyngrok --quiet")'


In [68]:
!pip install pyngrok

from pyngrok import ngrok
ngrok.set_auth_token("") # Add your auth token in here

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [69]:
!streamlit run /content/main.py --server.port 8000 & npx localtunnel --port 8000

/bin/bash: streamlit: command not found
npx: installed 22 in 3.284s
your url is: https://cruel-spoons-wash.loca.lt
^C
